# Processing NYC Taxi Data

In [6]:
#import libraries
# This ensures visualizations are plotted inside the notebook
%matplotlib inline
import io
import os              # This provides several system utilities
import pandas as pd    
import seaborn as sns 
import numpy as np
import rtree
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
from cenpy import products
import cenpy
import scipy.stats  as stats # low-level stats & probability
import statsmodels.formula.api as smf # high-level stats
import requests
import rasterio as rio
import pysal as ps
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import normalize
from sklearn.datasets import make_regression
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


from torch.autograd import Variable
import torch.nn.functional as F

In [23]:
def saveDatasetTaxiSnippet(hour_range=1):
    '''
    save first hour aggregate SI data from one week NYC Taxi data 
    '''
    #data cleaning
    taxi_data = pd.read_csv('yellow_tripdata_2019-05.csv')
    taxi_data["pickuptime"] = pd.to_datetime(taxi_data["tpep_pickup_datetime"])
    taxi_data["dropofftime"] = pd.to_datetime(taxi_data["tpep_dropoff_datetime"])
    taxi_data["hour"] = taxi_data["pickuptime"].map(lambda x: x.hour)
    taxi_data["weekday"] = taxi_data["pickuptime"].map(lambda x: x.weekday)
    taxi_data["weekday_name"] = taxi_data["pickuptime"].map(lambda x: x.day_name)
    taxi_lite = taxi_data[['PULocationID','DOLocationID','trip_distance','pickuptime','dropofftime','hour','weekday_name']]
    week_mask = (taxi_lite['pickuptime'] >= '2019-5-6') & (taxi_lite['pickuptime'] < '2019-5-13')
    #get sample data of one week
    sampleweek_taxi_lite = taxi_lite.loc[week_mask]
    #get hourly count
    hour_si = pd.DataFrame({'count' : sampleweek_taxi_lite.groupby(['PULocationID','DOLocationID','hour'] ).size()}).reset_index()
    hour_si = hour_si[hour_si['PULocationID']!=hour_si['DOLocationID']]
    hour_si = hour_si[ (hour_si['PULocationID']<264) & (hour_si['DOLocationID']<264)]
    # due to missing records of 264 and 265 in shp

    #get distances
    taxi_zone = gpd.read_file('taxi_zones/taxi_zones.shp')
    centroids =  ps.lib.weights.get_points_array_from_shapefile("taxi_zones/taxi_zones.shp")
    dist_mat = ps.lib.cg.distance_matrix(centroids)
    dist_mat

    for i, row in hour_si.iterrows():
        hour_si.at[i,'Dij'] = dist_mat[int(row['PULocationID']-1)][int(row['DOLocationID']-1)]
   
    #generate final dataframe containing O,D,Dist,flow
    df = pd.DataFrame()
    for hr in range(24):
        hour1 = hour_si[hour_si['hour'] == hr]

        res_do = pd.DataFrame(hour1.groupby('DOLocationID')['count'].count())
        res_do['hour'] = hr * np.ones(res_do.shape)
        res_do = res_do.astype({'hour': 'int32'})
        res_do = res_do.rename(columns = {'count':'Dj'})

        res_pu = pd.DataFrame(hour1.groupby('PULocationID')['count'].count())
        res_pu['hour'] = hr * np.ones(res_pu.shape)
        res_pu = res_pu.astype({'hour': 'int32'})
        res_pu = res_pu.rename(columns = {'count':'Oi'})

        hour1_O = pd.merge(hour1, res_pu,how = 'left',on=['PULocationID', 'hour'])
        hour1_OD = pd.merge(hour1_O, res_do,how = 'left',on=['DOLocationID', 'hour'])
        hour1_OD = hour1_OD.rename(columns={'count':'flow'})
        #print(hour1_OD.head())
        df = df.append(hour1_OD)
    df
    # find (PU,DO) that has hour [0,...,23]
    grouped = df.groupby(['PULocationID','DOLocationID'])

    df2 = grouped.aggregate(lambda x: np.array(x))
    df_full = df2[df2['hour'].apply(lambda x: x.size) ==24]
    df_clear_full = df_full.reset_index()[['Oi','Dj','Dij','flow']]
    
    #extract Y
    Y = df_clear_full['flow'].to_numpy()
    Y = np.stack(Y, axis=0)
    Y[0].reshape(24,-1,1)
    
    #extract Xs
    X_pd = df_clear_full[['Oi','Dj','Dij']]

    X = np.array([], dtype=np.int64).reshape(0,3)
    for i, row in X_pd.iterrows():
        X_row = np.array( [row['Oi'],row['Dj'],row['Dij']])
        X = np.vstack([X, X_row.T]) if X.size else X_row.T
        
    X_re1 = X.reshape(-1,24,3)
    X_d2 = X_re1[:1000,].reshape(-1,24*3)

    X_head = X_d2[:,0:(3*hour_range)]
    Y = Y[:1000,].reshape(-1,24,1,1)
    Y_d2 = Y.reshape(-1,24*1)
    Y_d2[:,-1].shape
    #Y_tail = Y_d2[:,-1]
    Y_head = Y_d2[:,0:hour_range]

    np.save('data/simple_taxi_X_{}'.format(hour_range),X_head)
    np.save('data/simple_taxi_Y_{}'.format(hour_range),Y_head)
    return

In [26]:
t0 = time.time()
saveDatasetTaxiSnippet(24)
t1 = time.time()
print('time elapsed: {}'.format(t1-t0))

time elapsed: 139.17199778556824


In [10]:
def loadDatasetTaxiTimeSeries(hour_range):
    X = np.load('data/simple_taxi_X_{}.npy'.format(hour_range))
    Y = np.load('data/simple_taxi_Y_{}.npy'.format(hour_range))
    return (X,Y)

In [15]:
def loadDatasetTaxiSnippet():
    X = np.load('data/simple_taxi_X.npy')
    Y = np.load('data/simple_taxi_Y.npy')
    return (X,Y)

# Prepare data for streamlit visualization

In [4]:
# download from https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-05.csv
taxi_data = pd.read_csv('yellow_tripdata_2019-05.csv')

In [5]:
taxi_data["pickuptime"] = pd.to_datetime(taxi_data["tpep_pickup_datetime"])
taxi_data["dropofftime"] = pd.to_datetime(taxi_data["tpep_dropoff_datetime"])
taxi_data["hour"] = taxi_data["pickuptime"].map(lambda x: x.hour)
taxi_data["weekday"] = taxi_data["pickuptime"].map(lambda x: x.weekday)
taxi_data["weekday_name"] = taxi_data["pickuptime"].map(lambda x: x.day_name)
taxi_lite = taxi_data[['PULocationID','DOLocationID','trip_distance','pickuptime','dropofftime','hour','weekday_name']]
week_mask = (taxi_lite['pickuptime'] >= '2019-5-6') & (taxi_lite['pickuptime'] < '2019-5-13')
#get sample data of one week
sampleweek_taxi_lite = taxi_lite.loc[week_mask]
#get hourly count
hour_si = pd.DataFrame({'count' : sampleweek_taxi_lite.groupby(['PULocationID','DOLocationID','hour'] ).size()}).reset_index()
hour_si = hour_si[hour_si['PULocationID']!=hour_si['DOLocationID']]
hour_si = hour_si[ (hour_si['PULocationID']<264) & (hour_si['DOLocationID']<264)]
# due to missing records of 264 and 265 in shp

#get distances
taxi_zone = gpd.read_file('taxi_zones/taxi_zones.shp')
centroids =  ps.lib.weights.get_points_array_from_shapefile("taxi_zones/taxi_zones.shp")
dist_mat = ps.lib.cg.distance_matrix(centroids)
dist_mat

for i, row in hour_si.iterrows():
    hour_si.at[i,'Dij'] = dist_mat[int(row['PULocationID']-1)][int(row['DOLocationID']-1)]
#generate final dataframe containing O,D,Dist,flow
df = pd.DataFrame()
for hr in range(24):
    hour1 = hour_si[hour_si['hour'] == hr]
    
    res_do = pd.DataFrame(hour1.groupby('DOLocationID')['count'].count())
    res_do['hour'] = hr * np.ones(res_do.shape)
    res_do = res_do.astype({'hour': 'int32'})
    res_do = res_do.rename(columns = {'count':'Dj'})
    
    res_pu = pd.DataFrame(hour1.groupby('PULocationID')['count'].count())
    res_pu['hour'] = hr * np.ones(res_pu.shape)
    res_pu = res_pu.astype({'hour': 'int32'})
    res_pu = res_pu.rename(columns = {'count':'Oi'})
    
    hour1_O = pd.merge(hour1, res_pu,how = 'left',on=['PULocationID', 'hour'])
    hour1_OD = pd.merge(hour1_O, res_do,how = 'left',on=['DOLocationID', 'hour'])
    hour1_OD = hour1_OD.rename(columns={'count':'flow'})
    #print(hour1_OD.head())
    df = df.append(hour1_OD)
df
# find (PU,DO) that has hour [0,...,23]
grouped = df.groupby(['PULocationID','DOLocationID'])

df2 = grouped.aggregate(lambda x: np.array(x))
df_full = df2[df2['hour'].apply(lambda x: x.size) ==24]
df_clear_full = df_full.reset_index()[['Oi','Dj','Dij','flow']]

Y = df_clear_full['flow'].to_numpy()
Y = np.stack(Y, axis=0)
Y[0].reshape(24,-1,1)

X_pd = df_clear_full[['Oi','Dj','Dij']]

In [8]:
df2.head()

hour flow      Dij Oi  Dj
PULocationID DOLocationID                          
1            138             4    1  88512.1  1  69
2            68             15    1  66998.1  1  63
             261            12    1  60620.8  1  53
3            32             11    1  4837.74  1   2
             39              4    1  83761.9  1   5

In [9]:
X = np.array([], dtype=np.int64).reshape(0,3)
for i, row in X_pd.iterrows():
    X_row = np.array( [row['Oi'],row['Dj'],row['Dij']])
#     print(X_row.T)
#     print(X_row.T.shape)
    X = np.vstack([X, X_row.T]) if X.size else X_row.T
#     if i > 2:
#         break
X_re1 = X.reshape(-1,24,3)

In [13]:
X.shape

(24144, 3)

In [186]:
taxi_zone = taxi_zone.to_crs({'init': 'epsg:4326'})
taxi_zone['clat'] = taxi_zone.geometry.centroid.y
taxi_zone['clon'] = taxi_zone.geometry.centroid.x
taxi_zone.head()
taxi_zone_lite = taxi_zone[['LocationID','clat','clon']]
taxi_zone_lite

,LocationID,clat,clon
0,1,40.691831,-74.174000
1,2,40.616745,-73.831299
2,3,40.864474,-73.847422
3,4,40.723752,-73.976968
4,5,40.552659,-74.188484
...,...,...,...
258,259,40.897932,-73.852215
259,260,40.744235,-73.906306
260,261,40.709139,-74.013023
261,262,40.775932,-73.946510


In [193]:
hour_si_location = pd.merge(hour_si, taxi_zone_lite, left_on='PULocationID', right_on='LocationID')

In [194]:
hour_si_location = pd.merge(hour_si_location, taxi_zone_lite, left_on='DOLocationID', right_on='LocationID',suffixes =('','_r'))

In [199]:

hour_si_location = hour_si_location[['hour','clat','clon','clat_r','clon_r','count']]
hour_si_location = hour_si_location.rename(columns={'clat':'lat','clon':'lon','clat_r':'lat2','clon_r':'lon2'})
hour_si_location.head()

,hour,lat,lon,lat2,lon2,count
0,4,40.691831,-74.174000,40.774376,-73.873629,1
1,6,40.723752,-73.976968,40.774376,-73.873629,1
2,7,40.723752,-73.976968,40.774376,-73.873629,1
3,8,40.723752,-73.976968,40.774376,-73.873629,2
4,9,40.723752,-73.976968,40.774376,-73.873629,1


In [200]:
hour_si_location.to_csv('hour_si_deckgl.csv',index=False)

In [204]:
data = hour_si_location
hour_sel = 1
trip_thres = 1
data = data[data.hour == hour_sel]
data = data[data['count'] > trip_thres]
data['colorR'] =np.random.uniform(size=data.shape[0], high=255.0)
data

,hour,lat,lon,lat2,lon2,count,colorR
526,1,40.646985,-73.786533,40.774376,-73.873629,4,161.048917
1251,1,40.709139,-74.013023,40.774376,-73.873629,2,94.597047
1311,1,40.723752,-73.976968,40.748428,-73.999917,5,95.529952
1432,1,40.782478,-73.965554,40.748428,-73.999917,2,94.404758
1454,1,40.712459,-73.998151,40.748428,-73.999917,4,166.117308
...,...,...,...,...,...,...,...
132177,1,40.762253,-73.989845,40.703546,-73.875736,2,144.016447
132220,1,40.646985,-73.786533,40.703546,-73.875736,2,151.779481
132453,1,40.804334,-73.951292,40.819676,-73.898956,2,121.143631
132545,1,40.748575,-73.985156,40.819676,-73.898956,2,189.385971


In [220]:
color_cat = pd.qcut(data['count'],10,duplicates= 'drop',labels=range(8))

In [223]:
data['color_idx'] = color_cat.cat.codes

In [224]:
data.head()

,hour,lat,lon,lat2,lon2,count,colorR,color_idx
526,1,40.646985,-73.786533,40.774376,-73.873629,4,161.048917,1
1251,1,40.709139,-74.013023,40.774376,-73.873629,2,94.597047,0
1311,1,40.723752,-73.976968,40.748428,-73.999917,5,95.529952,2
1432,1,40.782478,-73.965554,40.748428,-73.999917,2,94.404758,0
1454,1,40.712459,-73.998151,40.748428,-73.999917,4,166.117308,1


In [6]:
X_d2 = X_re1[:1000,].reshape(-1,24*3)
X_d2.shape

(1000, 72)

In [7]:
X_d2[:,0:3].shape

(1000, 3)

In [8]:
X_head = X_d2[:,0:3]

In [9]:
Y = Y[:1000,].reshape(-1,24,1,1)
Y.shape

Y_d2 = Y.reshape(-1,24*1)
Y_d2.shape

Y_d2

Y_d2[:,-1].shape
Y_tail = Y_d2[:,-1]

Y_head = Y_d2[:,0].reshape(-1,1)
Y_head.shape

(1000, 1)

In [11]:

normed_Y = normalize(Y_head, axis=0, norm='l1')


In [12]:
np.save('simple_X',X_head)
np.save('simple_Y',Y_head)